In [ ]:
# 先裝這個
# pip install webdriver_manager

In [10]:
from selenium.webdriver import Firefox
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
import pandas as pd
import time
import random
from bs4 import BeautifulSoup

In [11]:
df = pd.read_csv("/Users/jeanhong/Desktop/Finbert/article_selected2017.11.12.csv", header=0, encoding= 'ISO-8859-1')

使用Firefox瀏覽器來進行爬蟲，Chrome會搞事列入黑名單

In [12]:

driver = Firefox(service=Service(GeckoDriverManager().install()))


低配版在driver開啟的firefox進行手動登入</br>
執行完這個cell先登入email、pw後再執行下面程式碼

In [6]:
## email = 'mp3jean606@yahoo.com.tw'
## pw = 'G#RR@@pQZ@?F7rf'
news_url = "https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.wsj.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-wsj-223-2&nonce=1751d65e-1cff-45df-9662-6bfbfbc2daa4&state=B_rNPlZPv49OnM4Z.ZSFIoRp0ABdiAE1i4JIaDEYIAWQy8eYhWo3AGryShN8aNDuwuJGRkUxR7h9ISOfz&resource=https%253A%252F%252Fwww.wsj.com%252F%253Fmod%253Dwsjheader_logo&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin"
driver.get(news_url)

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:662:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:816:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


爬蟲開始

In [17]:
titles = []
sub_titles = []
articles = []
links = []

for link in df.url:
    print(link)

    driver.execute_script("window.open('');")  # 開啟新視窗
    driver.switch_to.window(driver.window_handles[1])  # 移動至新視窗
    
    try:
        driver.get(link) # 在新分頁中依序請求url(新聞網址)
    except Exception as e:
        print("訪問 URL 時出錯:", link)
        print("錯誤詳情:", e)
        continue  # 跳過這個 URL，繼續下一個

    html_text = driver.page_source # 用page_sorce將當前url頁面取回後放入等號左邊的html_text變數
    soup = BeautifulSoup(html_text, 'html.parser')

    # 使用錯誤處理來檢查元素是否存在
    title = soup.find('h1', 'css-1lvqw7f-StyledHeadline e1ipbpvp0') # 抓文章標題
    if title:
        titles.append(title.text)
    else:
        titles.append("無標題")

    sub_title = soup.find('h2', 'css-jiugt2-Dek-Dek e1jnru6p0') # 抓副標題
    if sub_title:
        sub_titles.append(sub_title.text)
    else:
        sub_titles.append("無副標題")

    contents = soup.find_all('p', 'css-k3zb6l-Paragraph e1e4oisd0') # 抓內文
    full_contents = ''
    if contents:
        for content in contents:
            full_contents += content.text
    else:
        full_contents = "無內文"

    articles.append(full_contents)
    links.append(link)

    driver.close() # 關起當前的視窗
    driver.switch_to.window(driver.window_handles[0])  # 切回最初的視窗
    delay_choices = [8, 5, 10, 6, 15, 11]  #延遲的秒數
    delay = random.choice(delay_choices)  #隨機選取秒數
    time.sleep(delay)  #延遲 


driver.quit()


https://www.wsj.com/articles/instagram-says-hack-that-targeted-celebrities-was-wider-than-previously-thought-1504309242
https://www.wsj.com/articles/august-proves-mixed-month-for-auto-makers-1504278622
https://www.wsj.com/articles/alexa-and-siri-escalate-battle-of-virtual-assistants-1504307382
https://www.wsj.com/articles/time-warner-cable-subscriber-data-exposed-on-cloud-server-1504303918
https://www.wsj.com/articles/when-the-truth-is-crazy-1504303308
https://www.wsj.com/articles/sometimes-stereotypes-can-be-a-helpful-formulation-1504302743
https://www.wsj.com/articles/googlers-how-free-do-you-feel-to-express-your-views-1504302690
https://www.wsj.com/articles/why-would-china-help-us-with-pakistan-north-korea-1504302706
https://www.wsj.com/articles/cuomos-fracking-opposition-hurts-new-york-1504302605
https://www.wsj.com/articles/economics-is-about-trade-offs-not-solutions-1504302589
https://www.wsj.com/articles/sec-drops-fraud-suit-against-wall-street-financier-1504300376
https://www.w

In [18]:
zipped_data = list(zip(links, titles, sub_titles, articles))
data = pd.DataFrame(zipped_data, columns=['link', 'title', 'sub_title', 'articles'])
data

,link,title,sub_title,articles
0,https://www.wsj.com/articles/instagram-says-ha...,Instagram Says Hack That Targeted Celebrities ...,Theft of email addresses and phone numbers als...,Social-media app Instagram said a hack it disc...
1,https://www.wsj.com/articles/august-proves-mix...,August Proves to Be a Mixed Month for Auto Makers,GM posts strongest sales rise of the year but ...,U.S. auto industry sales continued to slump in...
2,https://www.wsj.com/articles/alexa-and-siri-es...,Alexa and Siri Escalate Battle of Virtual Assi...,"Apple, Amazon, Google, Microsoft racing to mak...",Apple Inc. and Amazon.com Inc. AMZN 3.20%incre...
3,https://www.wsj.com/articles/time-warner-cable...,Time Warner Cable Subscriber Data Exposed on C...,About 600 gigabytes of data from range of comp...,Millions of personal records of Time Warner Ca...
4,https://www.wsj.com/articles/when-the-truth-is...,無標題,Trump is diagnosed with a severe case of confo...,Two things are intriguing about the “Trump is ...
...,...,...,...,...
2122,https://www.wsj.com/articles/china-and-south-k...,China and South Korea to Restore Relations Aft...,Agreement is likely to come as a relief in Was...,SEOUL—China and South Korea moved to put a lon...
2123,https://www.wsj.com/articles/apple-is-designin...,"Apple Is Designing iPhones, iPads That Would D...",Company considers building devices only with c...,"Apple Inc., locked in an intensifying legal fi..."
2124,https://www.wsj.com/articles/waymo-showcases-d...,Waymo Showcases Driverless Vans Without Humans...,A demonstration by the Alphabet-owned company ...,"ATWATER, Calif.—Waymo LLC, the driverless-car ..."
2125,https://www.wsj.com/articles/indias-top-paymen...,India’s Top Payment App Has Eyes on U.S. Market,Backed by the likes of China’s Alibaba and Jap...,SINGAPORE—India’s largest mobile-payment app—b...


In [19]:
data.to_csv('/Users/jeanhong/Desktop/Finbert/articles_2017.9.10.csv')